In [1]:
# Importing Libraries
import os
import shutil
import cv2
import imutils

In [0]:
# function for splitting the dataset into train/val/test folder
base_dir = 'BT/'

for CLASS in os.listdir(base_dir):
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(base_dir + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(base_dir + CLASS)):
            img = base_dir + CLASS + '/' + FILE_NAME
            if n < 5:
                #shutil.copy(src, dst)
                shutil.copy(img, 'BTD/TEST/' + CLASS.upper() + '/' + FILE_NAME)
            elif n < 0.8*IMG_NUM:
                shutil.copy(img, 'BTD/TRAIN/'+ CLASS.upper() + '/' + FILE_NAME)
            else:
                shutil.copy(img, 'BTD/VAL/'+ CLASS.upper() + '/' + FILE_NAME)

In [0]:
# Functio for Image Pre-processing->finds the largest contour and crops the image at extreme points
def img_pre_process(image_path):
    image = cv2.imread(image_path)
    #cv2.imshow('original', image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    #cv2.imshow('gray', gray)
    

    ## threshold the image, then perform a series of erosions + dilations to remove any small regions of noise
    thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    #cv2.imshow('binary', thresh)

    ## find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)

    ## determine the most extreme points along the contour
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])

    ## draw the outline of the object;
    #cv2.drawContours(image.copy(), [c], -1, (0, 255, 255), 2)
    #cv2.imshow("extreme_points", image)

    ## img crop at extreme points
    x = extLeft[0]
    w = extRight[0]
    y = extTop[1]
    h = extBot[1]
    img_cropped = image[y:h, x:w]
     
    return img_cropped
    #cv2.imshow('image_cropped', img_cropped)
    #cv2.waitKey(0)

In [7]:
# Old directories(from where data to be fetched for pre-processing)
TRAIN_DIR = 'BT/TRAIN/'
TEST_DIR = 'BT/TEST/'
VAL_DIR = 'BT/VAL/'

# New directories(where data will be placed after preprocessing)
TRAIN_new = 'BTD/TRAIN_new/'
TEST_new = 'BTD/TEST_new/'
VAL_new = 'BTD/VAL_new/'

In [0]:
# Performing pre=processing on TRAIN,TEST,VAL set
def preprocess_db(db_dir, new_dir):
    for CLASS in os.listdir(db_dir):
        for n, file_name in enumerate(os.listdir(db_dir + CLASS)):
            img_path = db_dir + CLASS + '/' + file_name
            cropped_img = img_pre_process(img_path)
            new_path = new_dir + CLASS + '/' + file_name
            plt.imsave(new_path, cropped_img)

preprocess_db(db_dir= TEST_DIR, new_dir=TEST_new)
preprocess_db(db_dir= TRAIN_DIR, new_dir=TRAIN_new)
preprocess_db(db_dir= VAL_DIR, new_dir=VAL_new)